In [0]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType
from pyspark.sql.functions import concat, regexp_extract, date_format, current_timestamp
from pyspark.sql.functions import when, col, to_timestamp, to_date, regexp_replace
from pyspark.sql.functions import sha2, concat_ws, coalesce, lit, trim, struct
from pyspark.sql.functions import to_json, expr, to_utc_timestamp
from pyspark.sql.types import StructType, ArrayType
from pyspark.sql.functions import transform
import re
from functions import create_table_if_not_exists, get_function, silver_scd2_transform
from functions import read_snapshot_windowed, silver_standard_transform, overwrite_table
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

settings = {
    "read_function": "functions.stream_read_table",
    "transform_function": "functions.silver_standard_transform",
    "write_function": "functions.stream_upsert_table",
    "upsert_function": "functions.upsert_microbatch",
    "src_table_name": "edsm.bronze.systemsWithCoordinates",
    "dst_table_name": "edsm.silver.systemsWithCoordinates",
    "build_history": "true",
    "business_key": [
        "id"
    ],
    "surrogate_key": [
        "name",
        "coords.x",
        "coords.y",
        "coords.z",
        "date"
    ],
    "data_type_map": {
        "date": "timestamp"
    },
    "readStreamOptions": {},
    "writeStreamOptions": {
        "mergeSchema": "false",
        "checkpointLocation": "/Volumes/edsm/silver/utility/systemsWithCoordinates7days/_checkpoints/",
        "delta.columnMapping.mode": "name"
    },
    "ingest_time_column": "derived_ingest_time"
}


df = read_snapshot_windowed(spark, settings)
df = silver_standard_transform(df, settings, spark)

overwrite_table(df, settings, spark)